CODE WITH GUI AND CAM VISUALISATION:
The visualization technique used here to generate the heatmap is commonly known as "Class Activation Map (CAM)" or "Grad-CAM" (Gradient-weighted Class Activation Mapping). 
It's based on the idea of using the gradients of the target class flowing into the final convolutional layer to produce a coarse localization map highlighting important regions in the input image for predicting the given class.
In the code provided, we're using the gradients of the predicted class with respect to the output feature map of the last convolutional layer to compute the heatmap. 
This heatmap indicates the regions in the input image that are most influential in the model's decision-making process for predicting the given class.

In [1]:
import tensorflow as tf
import gradio as gr
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load the model
loaded_model = tf.keras.models.load_model('efficientnetb1_model.h5')

def predict_image_with_heatmap(img):
    print("Predicting image...")
    
    # Resize the image to (256, 256) and convert to a 3D array
    img_copy = img.copy()
    img_3d = cv2.resize(img_copy, (256, 256))
    img_3d = np.array(img_3d).reshape(-1, 256, 256, 3)

    # Predict using the loaded model
    prediction = loaded_model.predict(img_3d)[0]

    class_labels = ["No Cancer", "Adenocarcinoma", "Squamous Cell Carcinoma"]

    # Get the index of the class with the highest probability
    predicted_class_index = np.argmax(prediction)
    
    if predicted_class_index < len(class_labels):
        predicted_class = class_labels[predicted_class_index]
        confidence_percentage = prediction[predicted_class_index] * 100
        
        # If confidence percentage is above 95%, show prediction, else return "Wrong Image"
        if confidence_percentage > 99.96:
            # Get the output of the last convolutional layer in the model
            last_conv_layer = loaded_model.layers[0].get_layer("top_activation")
            last_conv_layer_model = tf.keras.Model(loaded_model.layers[0].input, last_conv_layer.output)
            with tf.GradientTape() as tape:
                conv_outputs = last_conv_layer_model(img_3d)
                loss = tf.reduce_mean(conv_outputs[:, :, :, predicted_class_index])
            grads = tape.gradient(loss, conv_outputs)
            pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
            heatmap = tf.reduce_mean(tf.multiply(conv_outputs, pooled_grads), axis=-1)
            heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
            heatmap = heatmap.numpy()[0]
            
            # Apply heatmap to the original image
            heatmap = cv2.resize(heatmap, (img_copy.shape[1], img_copy.shape[0]))
            heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
            heatmap = cv2.addWeighted(heatmap, 0.5, img_copy, 0.5, 0)
            
            # Convert heatmap to RGB format for display
            heatmap_rgb = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
            
            return heatmap_rgb, f"Predicted: {predicted_class} (Confidence: {confidence_percentage:.2f}%)"
        else:
            return None, "Wrong Image"
    else:
        return None, "Wrong Image"

image = gr.inputs.Image(shape=(250, 250))
heatmap_output = gr.outputs.Image(type="numpy")
label = gr.outputs.Label()

gr.Interface(fn=predict_image_with_heatmap, inputs=image, outputs=[heatmap_output, label], debug=True).launch()


c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Jess John\AppData\Local\Temp\ipykernel_12484\572698079.py:58: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  image = gr.inputs.Image(shape=(250, 250))
C:\Users\Jess John\AppData\Local\Temp\ipykernel_12484\572698079.py:58: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  image = gr.inputs.Image(shape=(250, 250))
C:\Users\Jess John\AppData\Local\Temp\ipykernel_12484\572698079.py:59: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  heatmap_output = gr.outputs.Image(type="numpy")
C:\Users\Jess 

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Predicting image...
1/1 [==============================] - 2s 2s/step
Predicting image...
1/1 [==============================] - 0s 116ms/step
Predicting image...
1/1 [==============================] - 0s 145ms/step
Predicting image...
1/1 [==============================] - 0s 118ms/step
Predicting image...
1/1 [==============================] - 0s 125ms/step
Predicting image...
1/1 [==============================] - 0s 114ms/step
